# RAG 기본 구조 이해하기

## 1. 사전작업(Pre-processing) - 1~4 단계

![rag-1.png](./assets/rag-1.png)

![rag-1-graphic](./assets/rag-graphic-1.png)

사전 작업 단계에서는 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 하는 4단계를 진행합니다.

- 1단계 문서로드(Document Load): 문서 내용을 불러옵니다.
- 2단계 분할(Text Split): 문서를 특정 기준(Chunk) 으로 분할합니다.
- 3단계 임베딩(Embedding): 분할된(Chunk) 를 임베딩하여 저장합니다.
- 4단계 벡터DB 저장: 임베딩된 Chunk 를 DB에 저장합니다.

## 2. RAG 수행(RunTime) - 5~8 단계

![rag-2.png](./assets/rag-2.png)

![](./assets/rag-graphic-2.png)

- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의합니다. 리트리버는 검색 알고리즘이며(Dense, Sparse) 리트리버로 나뉘게 됩니다. Dense: 유사도 기반 검색, Sparse: 키워드 기반 검색
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성합니다. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됩니다. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있습니다.
- 7단계 LLM: 모델을 정의합니다.(GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성합니다.

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


## 환경설정


API KEY 를 설정합니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다. 이러한 애플리케이션이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 [LangSmith](https://smith.langchain.com)를 사용하는 것입니다.

LangSmith가 필수는 아니지만, 유용합니다. LangSmith를 사용하고 싶다면, 위의 링크에서 가입한 후, 로깅 추적을 시작하기 위해 환경 변수를 설정해야 합니다.


In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


## 전체 코드

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
import openai

# 단계 1: 문서 로드(Load Documents)

loader = DirectoryLoader(
    "../data/대출사기형/csv",
    glob="*.csv",
    loader_cls=CSVLoader,
)

loader_pdf = DirectoryLoader(
    "C:\\Users\\USER\\OneDrive\\MS school\\2차 프로젝트\\intent\\pdf",
    glob="*.pdf",
    loader_cls=PyMuPDFLoader,
    )

# loader1 = PyMuPDFLoader("(교재)안전한 금융생활을 위한 보이스피싱 대처 방법.pdf")
docs = loader.load()
docs1 = loader_pdf.load()
docs

In [7]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)


split_documents = text_splitter.split_documents(docs)
split_documents1 = text_splitter.split_documents(docs1)

# 단계 3: 임베딩(Embedding) 생성
# embeddings = AzureOpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

In [8]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
vectorstore1 = FAISS.from_documents(documents=split_documents1, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()
retriever1 = vectorstore1.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt1 = PromptTemplate.from_template(
    """You are an assistant for summarization tasks. 
following pieces of retrieved context is example of similar context. catch the keyword it to answer the question if exists. 
summarize the conversation transparently. don't judge anything. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)
prompt = PromptTemplate.from_template(
    """You are an assistant for fraud detection tasks. 
Use the following pieces of retrieved context to answer the question. 
Label and reason message from a user with an intent whether to fraud or not. advertising is not fraud.
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

In [9]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
chain1 = (
    {"context": retriever1, "question": RunnablePassthrough()}
    | prompt1
    | llm
    | StrOutputParser()
)

In [11]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = """A : 본인 성함 알겟습니다. 제가 첫번째는 이제 본인이 혹시라도 이제 깡통 계좌 발생으로 해서 본인이 금전적인 피해를 보네 보신 게 있는지에 대해서 제가 확인을 해봐야 되겠구요
B : 네
A : 아 아 그 두번째 같은 경우는 혹시라도 본인이 이 자금을 어 이제 조사를 피하기 위해서 어 다른 계좌로 돈을 바로 이체를 하는 게 아니냐 예를 들어서 깡통 계좌가 발생하더라도 앞으로는 제가 계좌 쪽을 조사를 진행을 해야 되는데 깡통 계좌가 갑자기 이자 발생을 하면 아예 이제 (삐-) 씨가 당황스럽고 피해를 봤다고 생각을 할 거에요. 그냥 저희 입장에서 봤을 때는 혹시 제가 (삐-) 계좌 추적을 피하기 위해서 한번 받고 하면 네 그분한테 계좌 이체를 시도를 한 게 아니냐 그러면 절차가 조금 더 복잡해줄 수 있어요. 계좌 발생하면.
B : 네 네
A : 아 그래서 깡통 계좌 내용에 대한 거는 이제 실질적으로 제가 설명드렸듯이 이 계좌 잔고가 얼마가 있던 상관 없습니다. 돈이 액수가 필요한게 아니고
B : 네 예
A : 한번 확인을 해도 출금 시도를 해봐야 진짜로 내가 원하는 만큼 돈이 출금이 되냐를 제가 확인할 수가 있다는 거예요. 출금을 했을때 진짜 본인이 원하는 대로 돈이 나오지 않았을 경우 당사자가 실질적으로 내가 어 이게 깡통 계좌구나하고 그때 좀 더 빼드릴 수가 있다는 겁니다. 아 그쵸 그래서 어 한번 좀 확인해봐야 되는 거는 본인이 직접적으로 이제 농협 은행권에 잠시 뒤 방문을 해줘야 될 것 같은데, 어 이거는 제가 그렇다고 제가 통장을 가지고 제가 가서 뭐 돈을 찾을 수 없잖아요
B : 네 그렇죠
A : 말이 안 돼 예 말이 안 됐기 때문에 본인이 직접적으로 이동을 해줘 갖고 일단은 뭐 은행권으로 방문을 하는 데 있어서 뭐 번거로움이 있을 거 같애요
B : 아니요 내일 방문 하면 될 것 같은데요
A : 내일이요?
B : 네

"""
response = chain1.invoke(question)
print(response)

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


# 대화 요약:
A는 B에게 깡통 계좌 발생으로 인해 금전적인 피해를 본 적이 있는지 확인하고, 자금 조사를 피하기 위해 다른 계좌로 돈을 이체한 적이 있는지 묻고 있다. B는 이에 대해 네라고 답하며, A는 계좌 잔고와 상관없이 출금 시도를 통해 실제로 돈이 출금되는지 확인해야 한다고 설명한다. A는 B에게 농협 은행을 방문해 직접 확인할 필요가 있다고 말하고, B는 내일 방문하겠다고 답한다.

# 관련 문서:
보이스피싱 피해 발생 시 대처 방안에 대한 설명이 포함된 문서가 제공된다. 피해 발생 시 계좌 지급 정지 절차와 이의제기 방법, 통장협박 피해 예방, 금융결제원의 계좌 통합 관리 서비스 등 여러 대처 방법이 설명되어 있다.

요약:
A는 B에게 깡통 계좌와 관련된 금전적 피해 여부와 다른 계좌로의 자금 이체 여부를 확인하고, B는 내일 농협 은행을 방문해 직접 확인하겠다고 답한다. 관련 문서에서는 보이스피싱 피해 발생 시 대처 방안과 예방 방법이 설명된다.


Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


In [12]:
response1 = chain.invoke(response)
print(response1)

#Answer: 
사기 의도가 있습니다. 이유는 A가 B에게 깡통 계좌와 관련된 금전적 피해 여부와 다른 계좌로의 자금 이체 여부를 확인하는 것은 보이스피싱과 관련된 전형적인 사기 수법 중 하나입니다. B가 내일 농협 은행을 방문해 직접 확인하겠다고 답한 점도, 사기범이 피해자를 안심시키기 위해 주로 사용하는 방법입니다. 또한, 관련 문서에 보이스피싱 피해 발생 시 대처 방안과 예방 방법이 설명된 점도 이 대화가 사기와 관련이 있음을 뒷받침합니다.


In [ ]:




question = "주거래 공공은행 월봉리 적금은요 각종 주거래 건수에 따라 우대금리 줍니다. 상품특집은요? 공과금 카드대금, 이체 등 각종 주거래 건수에 따라 우대금리 제공하는 월봉리 적금입니다. 가입 대상은요? 실명의 개인 또는 개인 사업주입니다. 가입 기간은요? 만 원에서 150만 원입니다. 이자는 어떻게 주나요? 만기 일시 지급식입니다. 적립 방법은요? 자유 적립식입니다. 금리는요 최대 연 1.00 프로입니다 만기 자동 예치되나요? 총 삼 회 자동 예치됩니다. 일부 해지되나요? 예금 가입 후 이 회까지 가능합니다. 세제 혜택은요? 비과세 종합 저축 가능합니다."

prompt2 = PromptTemplate.from_template(
    """You are an assistant for discriminating two speakers using Punctuation mark.  
 Label the speaker A,B. Answer in Korean. 

#Question: 
{question} 

#Answer:"""
)
chain2 = {"question": RunnablePassthrough()} | prompt2 | llm | StrOutputParser()

response = chain2.invoke(question)
print(response)

response1 = chain1.invoke(response)
print(response1)

response2 = chain.invoke(response1)
print(response2)

In [ ]:
# from sentence_transformers import util 

# text = """사기범 : 예 고객님 담당자 김성도 대리입니다.
# 피해자 : 예
# 사기범 : 지금 법무사님이 두분 배정되셨어요.
# 피해자 : 네 네
# 사기범 : 네 네 지금 뭐 입증되십니까?
# 피해자 : 예 예 지금 준비 하고 있어요.
# 사기범 : 예 그러세요. 잠시만요. 지금 일단 먼저 법무사님 한분 안내 좀 해 드리겠습니다. 메모 가능하십니까?
# 피해자 : 예 불러주세요
# 사기범 : 예 예 이거 어 OOO원입니다. 잠시만요. 일단 국민은행으로 배정되셨구요.
# 피해자 : 네 국민이요.
# 사기범 : OOOOO
# 피해자 : 불러볼게요
# 사기범 : 네
# 피해자 : OOOOO. 성함은 어떻게 돼요?
# 사기범 : 예 OOO 법무사 님이십니다
# 피해자 : OOO 예 아 지금 보내고 바로 전화 드릴게요.
# 사기범 : OOO입니다.
# 피해자 : 예, 입금 하고 바로 전화 드릴게요.
# 사기범 : OOO님은 먼저 구십팔 만 원이세요.
# 피해자 : 예 구십팔만원이요
# 사기범 : 예 나머지 차액 백오십 이만원을 네네 다른 법무사님 배정 받으셔야 됩니다. OOO 법무사님 오늘 이제 금요일 정시 퇴근하셔서 네 네 일단 OOO법무사님이 구십팔만 원이십니다.
# 피해자 : 예 알겠습니다. 또 돌아 가면요? 우선은 OOO 법무사님 구십팔만원 상환 처리하신 다음에 전화 좀 바로 좀 부탁드릴게요. 저도 법무사님 빠르게 지금 배정 요청 중에 있으니까요.
# 피해자 : 예 예
# 사기범 : 제가 지금 빠르게해서 금요일 여섯 시까지 영수증 확인 돼가지고 여섯 시 반에는 자금 수령 제가 문제 없도록 도움 드리겠습니다.
# 피해자 : 네 네 대리님도 수고 많으셨습니다.
# 사기범 : 네 그 차액부분 백오십 이만원 배정 받으시면 부탁좀 드리겠습니다.
# 피해자 : 예 예 알겠습니다.
# 사기범 : 네 네"""
# text2 = """피해자 : 여보세요?
# 사기범 : 네, 여보세요.
# 피해자 : 네.
# 사기범 : 네, 안녕하세요. 여기는 현대저축은행 자금회수팀에 OOO팀장입니다.
# 피해자 : 네네.
# 사기범 : 네, 전화받으시는분 성함이 OOO 맞으신가요?
# 피해자 : 네 맞습니다.
# 사기범 : 네, 안녕하세요. 전화드린건 다름이 아니고요,
# 피해자 : 네네.
# 사기범 : 은행측에서 일시상환 요청이 들어와서 연락드렸습니다.
# 피해자 : 네네.
# 사기범 : 고객님 혹시 국민은행을 통해서 상환을 하신다고 요청협조 해주신 선례가 따로 어떻게 되실까요?
# 피해자 : 아~저, 좀 아는분인데요.
# 사기범 : 아 네네.
# 피해자 : 네, 저 고금리 사용하지 말라고 해서요 상환할려고 해요
# 사기범 : 아 그 알고계시는분이 국내 은행쪽에 종사하고 계시는 직원 분이신가요?
# 피해자 : 네네.
# 사기범 : 아 그러신가요, 알겠습니다. 안내받기로는 5월 19일 금요일 날짜로 자금받아보신 부분, 2300만원 전액 완납상환 하신다고 안내 받았는데요.
# 피해자 : 네네.
# 사기범 : 이 부분 맞으신가요?
# 피해자 : 네.
# 사기범 : 아, 맞으시구요. 그러시면 고객님께서 자동이체 설정해주신 계좌로 자동이체 해가도록 도움드릴거구요.
# 피해자 : 네.
# 사기범 : 시각은 오후 1시경에 결제처리 도와드릴 수 있도록 진행해 드려보겠습니다. 이 부분 동의하시구요?
# 피해자 : 네.
# 사기범 : 네, 알겠습니다.
# 피해자 : 네.
# 사기범 : 다음에도 저희 현대저축은행 이용 부탁 드려보겠구요.
# 피해자 : 네.
# 사기범 : 저는 현재저축은행 자금회수팀에 OOO팀장이었습니다.
# 피해자 : 네, 알겠습니다.
# 사기범 : 네~에.
# """
# query_result = embeddings.embed_query(text)
# query_result2 = embeddings.embed_query(text2)

# # show only the first 100 characters of the stringified vector

# print(str(query_result)[:100] + "...")

# cos_scores = util.pytorch_cos_sim(query_result, query_result2)[0]
# cos_scores

NameError: name 'embeddings' is not defined